In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!mkdir -p /content/drive/MyDrive/summaries
!mkdir -p /content/drive/MyDrive/latencies

!pip install yt-dlp

!apt update && apt install ffmpeg -y

!pip install git+https://github.com/openai/whisper.git

!pip install transformers

!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.6 MB/s eta 0:00:00
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://arch

In [4]:
import os

if os.path.exists("yt_video.mp4"):
    os.remove("yt_video.mp4")
if os.path.exists("yt_audio.wav"):
    os.remove("yt_audio.wav")

url = "https://www.youtube.com/watch?v=qYNweeDHiyU&t=71s"

os.system(f'yt-dlp -f "best" -o "yt_video.%(ext)s" "{url}"')

print("Download complete")

Download complete


In [5]:
print(os.listdir())
from moviepy.editor import VideoFileClip

yt_vid= VideoFileClip("yt_video.mp4")
yt_vid.audio.write_audiofile("yt_audio.wav")

['.config', 'drive', 'yt_video.mp4', 'sample_data']


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



MoviePy - Writing audio in yt_audio.wav


MoviePy - Done.


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("yt_audio.wav")
cmp_text = result["text"]

def chunk_text(text, max_words=400):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        txt = ' '.join(words[i:i+max_words])
        chunks.append(txt)
    return chunks

chunks = chunk_text(cmp_text)
print(f"Total Chunks: {len(chunks)}")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 54.1MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



In [ ]:
from transformers import pipeline
import time

summarizer = pipeline("summarization",model="facebook/bart-large-cnn")
summary_bart= []
latencies_bart= []

for i in range(len(chunks)):
    start_time= time.time()
    result = summarizer(chunks[i], max_length=100, min_length=20)
    summary_bart.append(result[0]['summary_text'])
    end_time= time.time()
    latency= end_time - start_time
    latencies_bart.append(latency)

cmp_summary_bart= ' '.join(summary_bart)
print("Final Summary")
print(cmp_summary_bart)

total_latency= sum(latencies_bart)
num_chunks= len(latencies_bart)
avg_latency= total_latency/num_chunks

with open("/content/drive/MyDrive/summaries/Summary_Bart.txt","w") as f:
  f.write(cmp_summary_bart)

with open("/content/drive/MyDrive/latencies/latency_Bart.txt","w") as f:
  f.write("Latencies :"+ str(latencies_bart))
  f.write("Average Latencies :"+ str(avg_latency))

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Final Summary
Everybody's talking about artificial intelligence these days. Are they the same thing or are they different? And if so, what are those differences? And deep learning is another one that comes into play. We're going to take a look at that in this video and try to explain how all these technologies relate. Machine learning is, as its name implies, the machine is learning. I don't have to program it. I give it lots of information and it observes things. Machine learning really came along and became more popularized in this time frame. The next layer of our Venn diagram involves deep learning. With deep learning we use these things called neural networks. Neural networks are ways that in a computer we simulate and mimic the way the human brain works. Generative AI can generate new content out of existing information. This is where chat bots and all of these things come from. Other types of models are audio models, video models, and things like that. In the early days, AI's ad

In [ ]:
summarizer = pipeline("summarization",model="t5-base")
summary_t5= []
latencies_t5= []

for i in range(len(chunks)):
    start_time= time.time()
    result = summarizer(chunks[i], max_length=100, min_length=20)
    summary_t5.append(result[0]['summary_text'])
    end_time= time.time()
    latency= end_time - start_time
    latencies_t5.append(latency)

cmp_summary_t5= ' '.join(summary_t5)
print("Final Summary")
print(cmp_summary_t5)

total_latency= sum(latencies_t5)
num_chunks= len(latencies_t5)
avg_latency= total_latency/num_chunks

with open("/content/drive/MyDrive/summaries/Summary_t5.txt","w") as f:
  f.write(cmp_summary_t5)

with open("/content/drive/MyDrive/latencies/latency_t5.txt","w") as f:
  f.write("Latencies :"+ str(latencies_t5))
  f.write("Average Latencies :"+ str(avg_latency))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

Final Summary
we're going to try to explain how all these technologies relate and how we can use them . in this video, we'll look at how artificial intelligence, machine learning and deep learning fit together . a lot of myths and misconceptions have been about these three technologies . expert systems of the 1980s, maybe in the 90s became more popular . machine learning is, as its name implies, the machine is learning . the more training data you can give it, the more confident it can be in predicting . the most recent advancements in the field of artificial intelligence all really are in this space . foundation models are where we get some of these kinds of things . if you start typing something in and then it predicts what your next word will be, it'll predict what the next one will be . chat bots and deepfakes are examples of generative AI that can generate new content . john avlon: the possibilities of generating new content are what's gotten all of the attention . chatbots and al

In [ ]:
summarizer = pipeline("summarization",model="google/pegasus-xsum")
summary_pegasus= []
latencies_pegasus= []

for i in range(len(chunks)):
    start_time= time.time()
    result = summarizer(chunks[i], max_length=100, min_length=20)
    summary_pegasus.append(result[0]['summary_text'])
    end_time= time.time()
    latency= end_time - start_time
    latencies_pegasus.append(latency)

cmp_summary_pegasus= ' '.join(summary_pegasus)
print("Final Summary")
print(cmp_summary_pegasus)

total_latency= sum(latencies_pegasus)
num_chunks= len(latencies_pegasus)
avg_latency= total_latency/num_chunks

with open("/content/drive/MyDrive/summaries/Summary_pegasus.txt","w") as f:
  f.write(cmp_summary_pegasus)

with open("/content/drive/MyDrive/latencies/latency_pegasus.txt","w") as f:
  f.write("Latencies :"+ str(latencies_pegasus))
  f.write("Average Latencies :"+ str(avg_latency))

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


Final Summary
In this video, I'm going to try to answer some of the most frequently asked questions about artificial intelligence. Back when I was an undergrad riding a bike, we would use programming languages like C or C++. When I was growing up in the 1980s and 1990s, we used to do a lot of machine learning. In our series of interviews with leading scientists, Google's artificial intelligence chief, DeepMind's Amol Rajan, explains how chat bots and deepfakes are part of generative AI. In our series of letters from African-American journalists, film-maker, and columnist Richard Roeper looks at how artificial intelligence is changing the world.
